In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, timedelta
from datetime import datetime
from tqdm import tqdm
from dateutil.relativedelta import relativedelta
import statsmodels.api as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, timedelta
from tqdm import tqdm

In [2]:
df=pd.read_csv("returns.csv")

In [3]:
df.head()

,PERMNO,date,PRC,RET,SHROUT
0,10000,19851231,NaN,NaN,NaN
1,10000,19860131,-4.3750,C,3680.0
2,10000,19860228,-3.2500,-0.257143,3680.0
3,10000,19860331,-4.4375,0.365385,3680.0
4,10000,19860430,-4.0000,-0.098592,3793.0


In [4]:
df['PRC']=abs(df['PRC'])

In [5]:
df.dropna(inplace=True)

In [6]:
df = df[df.RET != 'C']

In [7]:
df['PERMNO'].nunique()

32442

In [8]:
df['date'] = df['date'].astype(str)
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')

In [9]:
df.set_index(df.date, inplace=True)
df.drop('date', axis=1, inplace=True)

In [10]:
df=df.sort_index()

In [11]:
df['PU']= pd.Series()

/Users/marcelo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


In [12]:
df.dtypes

PERMNO      int64
PRC       float64
RET        object
SHROUT    float64
PU        float64
dtype: object

In [13]:
df['RET'] = pd.to_numeric(df.RET, errors='coerce')

In [14]:
def trading_signal(full_permno_df):
    
    permno_id = full_permno_df.PERMNO.unique()[-1]
    
    if len(full_permno_df) <= 60:
        return []
    
    results = []
    for jump in range (60,len(full_permno_df)):
        permno_df=full_permno_df[jump-60:jump]
        list_final=permno_df.RET.sort_values()
        returns=[]
       
        for i in range(len(list_final)):
            returns.append(list_final[i])
        cum_prod=np.cumprod(1+np.array(returns))
        cum_prod=cum_prod[-1]**(1/60)-1
        results.append((permno_df.index[-1], permno_id, cum_prod))
        #results.append((permno_df.index[-1], permno_id, np.mean(returns)))
    return results

In [15]:
test_df=df.loc[df['PERMNO']==10001][60:120]
test=test_df['RET'].values+1

In [16]:
np.cumprod(test)[-1]**(1/60)-1

0.014622840295411166

In [17]:
np.cumprod(test)

array([1.012987  , 1.00129916, 1.04082444, 1.04082444, 1.12250938,
       1.08242008, 1.22941489, 1.24545015, 1.40790044, 1.59742496,
       1.58659442, 1.50452941, 1.20362353, 1.3021016 , 1.31595335,
       1.32980508, 1.31955494, 1.40378213, 1.45993342, 1.70222689,
       1.65967122, 1.6313008 , 1.60661922, 1.60661922, 1.63530862,
       1.65338368, 1.76940988, 1.75490603, 1.93474704, 1.9200894 ,
       1.86146139, 1.98458217, 2.04382393, 2.01420279, 2.19784975,
       2.09319034, 2.09319034, 2.08421892, 1.78215724, 1.90298216,
       2.10717595, 2.25987035, 2.19879283, 2.28307916, 2.1596696 ,
       2.06711264, 1.99800286, 1.93556527, 1.88483411, 1.89685369,
       1.89685369, 1.99169638, 2.11182953, 2.11182953, 2.04783476,
       2.13742753, 2.07265706, 2.26696866, 2.45480288, 2.38934066])

In [18]:
cum_prod=np.cumprod(1+np.array(test))-1
cum_prod[-1]

1.839820175239809e+18

In [19]:
grps = df.groupby('PERMNO')

# Comment out lines below to generate pickle

In [20]:
# grps = [x[1] for x in grps]

In [21]:
# import multiprocessing as mp

# with mp.Pool(mp.cpu_count()-1) as pool:
#     pool_result = pool.map(trading_signal, grps)

In [22]:
# import pickle

# with open('pool_results.pickle', 'wb') as f:
#     pickle.dump(pool_result, f)

In [23]:
with open('pool_results.pickle', 'rb') as f:
    mynewlist = pickle.load(f)

In [24]:
for i in range(len(mynewlist)):
    for j in range(len(mynewlist[i])):
        mynewlist[i][j]=(mynewlist[i][j][0].strftime('%Y-%m-%d'), mynewlist[i][j][1], mynewlist[i][j][2])
    

In [25]:
new_data = [y for x in mynewlist for y in x if len(x) > 0]

In [26]:
new_df = pd.DataFrame(new_data, columns=['time', 'permno', 'Avg_Return'])

In [27]:
len(new_df)

2431828

In [28]:
len(new_df.loc[new_df.Avg_Return < 0])

790749

In [29]:
df['time'] = pd.to_datetime(df.index)
new_df['time'] = pd.to_datetime(new_df['time'])

In [30]:
df.columns

Index(['PERMNO', 'PRC', 'RET', 'SHROUT', 'PU', 'time'], dtype='object')

In [31]:
final_df = df.merge(new_df, left_on=['time', 'PERMNO'], right_on=['time', 'permno'], how='left')

In [32]:
np.mean(np.isnan(final_df.Avg_Return))

0.3914968684394244

In [33]:
final_df.drop(['PU','permno'],axis=1,inplace=True)

In [34]:
final_df=final_df.dropna()
final_df.head()

,PERMNO,PRC,RET,SHROUT,time,Avg_Return
92535,18390,7.375,-0.132353,649.0,1964-12-31,-0.007398
92537,16512,29.250,0.005957,307.0,1964-12-31,0.004004
92538,23827,38.625,0.020197,7334.0,1964-12-31,0.012642
92539,26606,24.750,-0.019802,2533.0,1964-12-31,0.009751
92541,25048,37.875,0.016779,1312.0,1964-12-31,0.006575


In [35]:
final_df=final_df.sort_values(['time', 'Avg_Return'], ascending=[True, True])

In [36]:
final_df['RET']=round(final_df['RET'],4)
final_df['Avg_Return']=round(final_df['Avg_Return'],4)


In [37]:
new_array=np.array(final_df['time'].unique())

In [38]:
df_temp=final_df[final_df['time']==new_array[0]]
x=pd.qcut(df_temp['Avg_Return'],10,retbins=True)
array_bins=np.array(list(x[1]))
array_bins

array([-0.0236 , -0.00544, -0.0011 ,  0.0018 ,  0.0046 ,  0.0066 ,
        0.0086 ,  0.0106 ,  0.01288,  0.0153 ,  0.0329 ])

In [39]:
list_portfolios=['P1','P2','P3','P4','P5','P6','P7','P8','P9','P10']
d={}
for name in list_portfolios:
    d[name] = pd.DataFrame()

In [40]:
j=0
for name in d.keys():
        d[name]=df_temp[(df_temp.Avg_Return > array_bins[j]) & (df_temp.Avg_Return <= array_bins[j+1])]
        j+=1

In [41]:
for ele in list_portfolios:
    d[ele]=pd.DataFrame(d[ele])

In [42]:
for i in tqdm(range(1,len(new_array)-1)):
    df_temp=final_df[final_df['time']==new_array[i]]
    x=pd.qcut(df_temp['Avg_Return'],10,retbins=True)
    array_bins=np.array(list(x[1]))
    portfolio={}
    for name in list_portfolios:
        portfolio[name] = pd.DataFrame()
        j=0
    for name in portfolio.keys():
        portfolio[name]=df_temp[(df_temp.Avg_Return > array_bins[j]) & (df_temp.Avg_Return <= array_bins[j+1])]
        d[name]=pd.concat([d[name],portfolio[name]])
        j+=1
        

100%|██████████| 646/646 [01:05<00:00,  5.22it/s]


In [43]:
def portfolio_return(name):
    list_returns=[]
    for name in list_portfolios:
        list_returns.append(np.mean(d[name].Avg_Return))
    return list_returns

In [44]:
portfolio_returns=portfolio_return(list_portfolios)

In [45]:
portfolio_returns

[-0.02964936658911898,
 -0.01080884058685401,
 -0.0035160297919487283,
 0.0011047808715912019,
 0.004537550301603993,
 0.007472198689209516,
 0.010328523557791637,
 0.013493313677832852,
 0.017810585851554542,
 0.028228891642361576]

In [46]:
def portfolio_std(name):
    list_std=[]
    for name in list_portfolios:
        list_std.append(np.std(d[name].Avg_Return))
    return list_std

In [47]:
portfolio_stds=portfolio_std(list_portfolios)
portfolio_stds

[0.016945555202307984,
 0.008276913917949184,
 0.006661500164574347,
 0.005718301039786126,
 0.00519201212146932,
 0.00507877372126346,
 0.005225200389008275,
 0.0056121099408109314,
 0.0062213134990080216,
 0.01018580805595413]

In [48]:
market_df=pd.read_csv("rm_rf.CSV",index_col=0)

In [49]:
market_df.tail()

,rm_rf,rf
201804,0.0029,0.0014
201805,0.0265,0.0014
201806,0.0048,0.0014
201807,0.0319,0.0016
201808,0.0344,0.0016


In [50]:
d['P1'].tail()

,PERMNO,PRC,RET,SHROUT,time,Avg_Return
3981318,81116,9.5100,-0.0722,47793.0,2018-10-31,-0.0185
3976046,92536,0.7589,-0.0596,225805.0,2018-10-31,-0.0185
3975678,10180,6.6700,-0.4861,125492.0,2018-10-31,-0.0185
3978569,91319,13.5900,-0.1872,23800.0,2018-10-31,-0.0184
3980046,75033,5.1000,-0.0192,21938.0,2018-10-31,-0.0184


In [51]:
for ele in list_portfolios:
    d[ele]=d[ele][(d[ele]['time'] < '2018-9')]

In [52]:
len(d['P1'].time.unique())

645

In [53]:
len(market_df)

1106

In [54]:
market_df=market_df.loc['196412':'201808']

In [55]:
def capm(name):
    d_portfolio=d[name].groupby('time').mean()
    d_portfolio['rm_rf']=market_df['rm_rf'].values
    d_portfolio['rf']=market_df['rf'].values
    d_portfolio['RET']=d_portfolio['RET']-d_portfolio['rf']
    Y=d_portfolio['RET']
    X=d_portfolio['rm_rf']
    X_standard = sm.add_constant(X)
    model = sm.OLS(Y,X_standard)
    results = model.fit()
    print(results.summary())
    return results.params

In [56]:
list_coeffs=[]
for ele in list_portfolios:
    list_coeffs.append(capm(ele))

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.419
Model:                            OLS   Adj. R-squared:                  0.419
Method:                 Least Squares   F-statistic:                     464.6
Date:                Tue, 05 May 2020   Prob (F-statistic):           5.73e-78
Time:                        16:07:56   Log-Likelihood:                 842.37
No. Observations:                 645   AIC:                            -1681.
Df Residuals:                     643   BIC:                            -1672.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0249      0.003     -9.573      0.0

In [57]:
list_coeffs=pd.DataFrame(list_coeffs)

In [58]:
list_coeffs.rename(columns={"const":"CAPM alpha","rm_rf":"Beta"},inplace=True)

In [59]:
list_coeffs.insert(loc=0, column='Portfolio', value=list_portfolios)

In [60]:
list_coeffs.insert(loc=1, column='Portfolio Monthly return', value=portfolio_returns)

In [61]:
list_coeffs.insert(loc=2, column='Portfolio Monthly volatility', value=portfolio_stds)

In [62]:
list_coeffs

,Portfolio,Portfolio Monthly return,Portfolio Monthly volatility,CAPM alpha,Beta
0,P1,-0.029649,0.016946,-0.024920,1.260916
1,P2,-0.010809,0.008277,-0.006584,1.157664
2,P3,-0.003516,0.006662,-0.002673,1.071782
3,P4,0.001105,0.005718,0.000352,0.995818
4,P5,0.004538,0.005192,0.002595,0.947055
5,P6,0.007472,0.005079,0.004626,0.902592
6,P7,0.010329,0.005225,0.007331,0.900809
7,P8,0.013493,0.005612,0.010660,0.917009
8,P9,0.017811,0.006221,0.015402,1.023966
9,P10,0.028229,0.010186,0.025470,1.226383


In [63]:
list_coeffs_annualized=((1+list_coeffs['Portfolio Monthly return'])**12)-1

In [64]:
list_coeffs_annualized=pd.DataFrame(list_coeffs_annualized)

In [65]:
list_coeffs_annualized.insert(loc=0, column='Portfolio', value=list_portfolios)

In [66]:
list_coeffs_annualized['Portfolio Annual volatility']=list_coeffs['Portfolio Monthly volatility']*np.sqrt(12)

In [67]:
list_coeffs_annualized['Annual CAPM alpha']= list_coeffs['CAPM alpha']*12

In [68]:
list_coeffs_annualized.rename(columns={"Portfolio Monthly return":"Portfolio Annual Return"},inplace=True)

In [69]:
list_coeffs_annualized

,Portfolio,Portfolio Annual Return,Portfolio Annual volatility,Annual CAPM alpha
0,P1,-0.303142,0.058701,-0.299044
1,P2,-0.122266,0.028672,-0.079006
2,P3,-0.041386,0.023076,-0.032077
3,P4,0.013338,0.019809,0.004225
4,P5,0.055830,0.017986,0.031143
5,P6,0.093445,0.017593,0.055511
6,P7,0.131231,0.018101,0.087967
7,P8,0.174494,0.019441,0.127915
8,P9,0.235958,0.021551,0.184820
9,P10,0.396618,0.035285,0.305637


# Step 2

In [70]:
d_long_short_portfolio=d['P1'].groupby('time').mean().RET-d['P10'].groupby('time').mean().RET

In [71]:
d_long_short_portfolio=pd.DataFrame(d_long_short_portfolio)

In [72]:
d_long_short_portfolio.head()

,RET
time,
1964-12-31,-0.033721
1965-01-29,0.009317
1965-02-26,-0.017378
1965-03-31,-0.017939
1965-04-30,-0.048415


In [73]:
portfolio_return=np.mean(d_long_short_portfolio['RET'])
print("Long-Short Portfolio average return:",portfolio_return)

Long-Short Portfolio average return: -0.05020869235390338


In [74]:
d_long_short_portfolio['rm_rf']=market_df['rm_rf'].values

In [75]:
d_long_short_portfolio['rf']=market_df['rf'].values

In [76]:
d_long_short_portfolio['RET-rf']=d_long_short_portfolio['RET']-d_long_short_portfolio['rf'].values

In [77]:
Y=d_long_short_portfolio['RET']
X=d_long_short_portfolio['rm_rf']
X_standard = sm.add_constant(X)
model = sm.OLS(Y,X_standard)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.3587
Date:                Tue, 05 May 2020   Prob (F-statistic):              0.549
Time:                        16:07:56   Log-Likelihood:                 851.67
No. Observations:                 645   AIC:                            -1699.
Df Residuals:                     643   BIC:                            -1690.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0504      0.003    -19.638      0.0

In [78]:
alpha=results.params[0]
print("Long-Short Portfolio average return:",portfolio_return, "and alpha is:",alpha)

Long-Short Portfolio average return: -0.05020869235390338 and alpha is: -0.050390034027993115


# Step 3

In [79]:
fama_french=pd.read_csv("Portfolios_Formed_on_BE-ME.CSV")

In [80]:
fama_french.rename(columns={"Unnamed: 0" : "time"},inplace=True)

In [81]:
fama_french.set_index('time',inplace=True)

In [82]:
fama_french['Value_portf']=(fama_french['Hi 10']-fama_french['Lo 10'])/100

In [83]:
fama_french=fama_french.loc['196412':'201808']

In [84]:
fama_french['rm_rf']=market_df['rm_rf'].values

In [85]:
fama_french['Value_portf']=fama_french['Value_portf']#-fama_french['rm_rf']

In [86]:
fama_french.head()

,<= 0,Lo 30,Med 40,Hi 30,Lo 20,Qnt 2,Qnt 3,Qnt 4,Hi 20,Lo 10,...,3-Dec,4-Dec,5-Dec,6-Dec,7-Dec,8-Dec,9-Dec,Hi 10,Value_portf,rm_rf
time,,,,,,,,,,,,,,,,,,,,,
196412,-12.01,1.11,-0.33,-1.93,0.83,1.13,-1.01,-1.32,-1.25,1.42,...,1.89,0.25,0.33,-2.36,-0.15,-2.66,-0.44,-2.49,-0.0391,0.0003
196501,18.00,4.01,3.05,5.04,4.88,1.76,3.84,4.35,5.39,5.84,...,1.64,1.91,3.17,4.53,4.09,4.66,4.67,6.51,0.0067,0.0354
196502,13.70,0.65,0.32,1.14,0.36,0.45,1.38,0.83,0.97,0.50,...,1.46,-0.74,1.56,1.20,0.40,1.32,-0.16,2.71,0.0221,0.0044
196503,-2.44,-1.11,-1.45,0.63,-1.21,-1.46,-1.13,-0.02,0.93,-1.07,...,-0.83,-2.22,-1.35,-0.90,-0.29,0.31,0.49,1.59,0.0266,-0.0134
196504,15.16,3.79,2.63,3.40,4.38,2.07,2.96,2.85,4.56,5.30,...,2.16,1.96,2.16,3.78,3.48,2.12,6.05,2.35,-0.0295,0.0311


In [87]:
Y=fama_french['Value_portf'].values
X=d_long_short_portfolio['RET'].values
X_standard = sm.add_constant(X)
model = sm.OLS(Y,X_standard)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.175
Model:                            OLS   Adj. R-squared:                  0.174
Method:                 Least Squares   F-statistic:                     136.7
Date:                Tue, 05 May 2020   Prob (F-statistic):           9.16e-29
Time:                        16:07:57   Log-Likelihood:                 1112.8
No. Observations:                 645   AIC:                            -2222.
Df Residuals:                     643   BIC:                            -2213.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0196      0.002      9.096      0.0

In [88]:
Y=d_long_short_portfolio['RET'].values
X=fama_french[["Value_portf",'rm_rf']] 
X_standard = sm.add_constant(X)
model = sm.OLS(Y,X_standard)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.175
Model:                            OLS   Adj. R-squared:                  0.173
Method:                 Least Squares   F-statistic:                     68.24
Date:                Tue, 05 May 2020   Prob (F-statistic):           1.34e-27
Time:                        16:07:57   Log-Likelihood:                 913.65
No. Observations:                 645   AIC:                            -1821.
Df Residuals:                     642   BIC:                            -1808.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -0.0525      0.002    -22.454      

In [89]:
np.corrcoef(d_long_short_portfolio['RET'].values,fama_french['Value_portf'])

array([[1.        , 0.41868795],
       [0.41868795, 1.        ]])